In [1]:
# Faster RCNN network to find and classify cells in images with Pytorch


# Imports
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.utils as utils
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
import time
import copy
import random
import math
import cv2
import pandas as pd


ImportError: cannot import name 'Path' from 'os' (/Users/deniz/opt/anaconda3/envs/comp411/lib/python3.7/os.py)

In [3]:
# read in the csv file as a pandas dataframe
df = pd.read_csv("QC/df_train.csv")
df.head()

,image_id,super_classification,xmin,ymin,xmax,ymax
0,TCGA-AR-A0U4-DX1_id-5ea40a88ddda5f8398990ccf_l...,4,135,83,176,123
1,TCGA-AC-A6IW-DX1_id-5ea40997ddda5f83989810a2_l...,2,149,258,164,275
2,TCGA-A2-A0YE-DX1_id-5ea40a24ddda5f839898afb8_l...,3,63,276,93,293
3,TCGA-GM-A2DH-DX1_id-5ea40adcddda5f83989951a2_l...,4,204,88,233,103
4,TCGA-E2-A1LH-DX1_id-5ea40a58ddda5f839898e0ed_l...,1,35,203,76,243


In [ ]:
# Define the dataset class where the inputs are:
# df: a dataframe that contains the image id, classification and xmax, xmin, ymin, ymax for the bounding box
# image_dir and transforms
class CellDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        super().__init__()
        self.df = df
        self.image_ids = self.df["image_id"].unique()
        self.image_dir = os.path(image_dir)
        self.transforms = transforms
        
    def __len__(self):
        return self.image_ids.shape[0]
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df["image_id"] == image_id]

        # get the image
        image_name = image_id + ".png"
        image = Image.open(os.path.join(self.image_dir, image_name)).convert("RGB")
        image = transforms.ToTensor()(image)

        # get the bounding box coordinates
        boxes = records[["xmin", "ymin", "xmax", "ymax"]].values
        boxes = torch.as_tensor(boxes, dtype=torch.int64)


        # get the classification
        labels = torch.tensor(records["super_classification"].values, dtype=torch.int64)
        

        # create a target dictionary
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])


        # apply the transformations
        if self.transforms is not None:
            image = self.transforms(image)
            
        return image, target, image_id
    



